In [1]:
import os
import nibabel as nib
import numpy as np
from skimage import morphology

# --- CONFIGURACIÓN (Ajusta si tus carpetas se llaman distinto) ---
input_folder = r"C:\Users\marta\Desktop\Salida_Predicciones"
output_folder = r"C:\Users\marta\Desktop\Salida_Predicciones_Limpias_3px"

# "min_size=3" significa: Mantén todo lo que sea >= 3. Borra lo que sea < 3.
# (Es decir, elimina 1 y 2).
min_size = 3  

os.makedirs(output_folder, exist_ok=True)

print(f"Iniciando limpieza (eliminando lesiones < {min_size} vóxeles)...")

files = [f for f in os.listdir(input_folder) if f.endswith(".nii.gz")]

for filename in files:
    # 1. Cargar imagen original
    file_path = os.path.join(input_folder, filename)
    img_nii = nib.load(file_path)
    data = img_nii.get_fdata()
    
    # 2. Convertir a booleano para la función de limpieza
    mask_bool = data > 0
    
    # 3. Aplicar el filtro
    # remove_small_objects borra islas conexas más pequeñas que 'min_size'
    cleaned_bool = morphology.remove_small_objects(mask_bool, min_size=min_size)
    
    # 4. Volver a convertir a formato numérico (0 y 1)
    cleaned_data = cleaned_bool.astype(np.uint8) 
    
    # Reporte rápido por consola para que veas qué está haciendo
    diff = np.sum(mask_bool) - np.sum(cleaned_bool)
    if diff > 0:
        print(f"  > {filename}: Se borraron {diff} vóxeles de ruido.")
    
    # 5. Guardar manteniendo la geometría original (IMPORTANTE)
    new_img = nib.Nifti1Image(cleaned_data, img_nii.affine, img_nii.header)
    nib.save(new_img, os.path.join(output_folder, filename))

print("\n¡Limpieza terminada!")
print(f"Predicciones limpias guardadas en: {output_folder}")

Iniciando limpieza (eliminando lesiones < 3 vóxeles)...
  > sub-105.nii.gz: Se borraron 1 vóxeles de ruido.
  > sub-216.nii.gz: Se borraron 1 vóxeles de ruido.
  > sub-218.nii.gz: Se borraron 4 vóxeles de ruido.
  > sub-220.nii.gz: Se borraron 3 vóxeles de ruido.
  > sub-230.nii.gz: Se borraron 3 vóxeles de ruido.
  > sub-307.nii.gz: Se borraron 2 vóxeles de ruido.

¡Limpieza terminada!
Predicciones limpias guardadas en: C:\Users\marta\Desktop\Salida_Predicciones_Limpias_3px
